In [1]:
import pandas as pd
import numpy as np
import sys
import sklearn
print(pd.__version__)
print(np.__version__)
print(sys.version)
print(sklearn.__version__)

from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error, roc_curve, classification_report,auc)
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.ensemble import VotingClassifier

2.0.1
1.24.2
3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]
1.2.0


# Dataset Preprocessing

In dataset preprocessing we didn't make any changes. All preprocessing parts have been preserved.

In [2]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

# KDDTrain+_2.csv & KDDTest+_2.csv are the datafiles without the last column about the difficulty score
# these have already been removed.
df = pd.read_csv("KDDTrain+_2.csv", header=None, names = col_names)
df_test = pd.read_csv("KDDTest+_2.csv", header=None, names = col_names)

print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)

Dimensions of the Training set: (125973, 42)
Dimensions of the Test set: (22544, 42)


In [3]:
df.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [4]:
df.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,125973.00000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,...,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,287.14465,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,0.001222,0.395736,0.279250,...,182.148945,115.653005,0.521242,0.082951,0.148379,0.032542,0.284452,0.278485,0.118832,0.120240
std,2604.51531,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,0.045239,0.489010,23.942042,...,99.206213,110.702741,0.448949,0.188922,0.308997,0.112564,0.444784,0.445669,0.306557,0.319459
min,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,82.000000,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,63.000000,0.510000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,255.000000,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000
max,42908.00000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,5.000000,1.000000,7479.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
print('Label distribution Training set:')
print(df['label'].value_counts())

Label distribution Training set:
label
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: count, dtype: int64


In [6]:
print('Label distribution Test set:')
print(df_test['label'].value_counts())

Label distribution Test set:
label
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail             14
rootkit              13
xterm                13
teardrop             12
xlock                 9
land                  7
xsnoop                4
ftp_write             3
worm                  2
loadmodule            2
perl                  2
sqlattack             2
udpstorm              2
phf                   2
imap                  1
Name: count, dtype: int64


In [7]:
# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).

# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

#see how distributed the feature service is, it is evenly distributed and therefore we need to make dummies for all.
print()
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())

Training set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'label' has 23 categories

Distribution of categories in service:
service
http        40338
private     21853
domain_u     9043
smtp         7313
ftp_data     6860
Name: count, dtype: int64


In [8]:
# Test set
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

# Test dataset has 6 missing data in service feature
# and 15 extra categories in label feature but the labels will be examined on their own. 
# so we just create dummies for the service feature.

Test set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 64 categories
Feature 'flag' has 11 categories
Feature 'label' has 38 categories


In [9]:
# insert code to get a list of categorical columns into a variable, categorical_columns
categorical_columns=['protocol_type', 'service', 'flag']
 # Get the categorical values into a 2D numpy array
df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()

,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF


### Dummy column preparation

In [10]:
# for each categorical variables in train dataset, we create dummy varibles, i.e. we create a new column for each category in the categorical variable.

# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2

print ("Dummies in train datasets:")
print(dumcols)

# do same for service feature of test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2
print()
print("Dummies count: ", len(dumcols))

Dummies in train datasets:
['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs', 'service_exec', 'service_finger', 'service_ftp', 'service_ftp_data', 'service_gopher', 'service_harvest', 'service_hostnames', 'service_http', 'service_http_2784', 'service_http_443', 'service_http_8001', 'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell', 'service_ldap', 'service_link', 'service_login', 'service_mtp', 'service_name', 'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn', 'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump', 'service_pop_2', 'service_pop_3', 'service_printer', 'service_private', 'service_red_i', 

In [11]:
# ecoding of the categorical values, each categorical value will be replaced by a unique number

# train set
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)

# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

In [12]:
df_categorical_values.head()

,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF


In [13]:
df_categorical_values_enc.head()

,protocol_type,service,flag
0,1,20,9
1,2,44,9
2,1,49,5
3,1,24,9
4,1,24,9


In [14]:
#In the dataset that has been label encoded, one-hot encoding is applied to create a new column for each category using predefined dummy column names. 
# The column values are filled in as binary (0s and 1s).

# For example, if the protocol_type value in the 0th row is "tcp," then only the Protocol_type_tcp column is assigned a value of 1, 
# while the other protocol type columns get a value of 0.

# train set
enc = OneHotEncoder()
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)

# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()
# 84 new dummies

,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# there was 6 missing data in the service feature of the test set, 
# so we need to add these columns to the test set and fill them with 0s

trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference # missing 6 service feature in test set

['service_urh_i',
 'service_aol',
 'service_red_i',
 'service_http_8001',
 'service_harvest',
 'service_http_2784']

In [16]:
for col in difference:
    testdf_cat_data[col] = 0

testdf_cat_data.shape
# now test set has 84 dummy columns as well

(22544, 84)

In [17]:
# merge the encoded dummy columns for the categorical data with the original dataset and drop the categorical columns

# train set
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)

# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)

print(newdf.shape)
print(newdf_test.shape)

(125973, 123)
(22544, 123)


### Splitting Dataset For Each Attack

In [18]:
# take label column
labeldf=newdf['label']
labeldf_test=newdf_test['label']

# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})

newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})

# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test

print(newdf['label'].head())

0    0
1    0
2    1
3    0
4    0
Name: label, dtype: int64


In [19]:
# prepare which values will be dropped from the each attack dataset
to_drop_DoS = [2,3,4]
to_drop_Probe = [1,3,4]
to_drop_R2L = [1,2,4]
to_drop_U2R = [1,2,3]

# prepare the datasets for the attacks
DoS_df=newdf[~newdf['label'].isin(to_drop_DoS)]
Probe_df=newdf[~newdf['label'].isin(to_drop_Probe)]
R2L_df=newdf[~newdf['label'].isin(to_drop_R2L)]
U2R_df=newdf[~newdf['label'].isin(to_drop_U2R)]

#test
DoS_df_test=newdf_test[~newdf_test['label'].isin(to_drop_DoS)]
Probe_df_test=newdf_test[~newdf_test['label'].isin(to_drop_Probe)]
R2L_df_test=newdf_test[~newdf_test['label'].isin(to_drop_R2L)]
U2R_df_test=newdf_test[~newdf_test['label'].isin(to_drop_U2R)]

print('Train:')
print('Dimensions of DoS:' ,DoS_df.shape)
print('Dimensions of Probe:' ,Probe_df.shape)
print('Dimensions of R2L:' ,R2L_df.shape)
print('Dimensions of U2R:' ,U2R_df.shape)
print('Test:')
print('Dimensions of DoS:' ,DoS_df_test.shape)
print('Dimensions of Probe:' ,Probe_df_test.shape)
print('Dimensions of R2L:' ,R2L_df_test.shape)
print('Dimensions of U2R:' ,U2R_df_test.shape)

Train:
Dimensions of DoS: (113270, 123)
Dimensions of Probe: (78999, 123)
Dimensions of R2L: (68338, 123)
Dimensions of U2R: (67395, 123)
Test:
Dimensions of DoS: (17171, 123)
Dimensions of Probe: (12132, 123)
Dimensions of R2L: (12596, 123)
Dimensions of U2R: (9778, 123)


In [20]:
# Split dataframes into X & Y
# assign X as a dataframe of feautures and Y as a series of outcome variables
# X input, Y expected output

X_DoS = DoS_df.drop('label',axis=1)
Y_DoS = DoS_df.label
X_Probe = Probe_df.drop('label',axis=1)
Y_Probe = Probe_df.label
X_R2L = R2L_df.drop('label',axis=1)
Y_R2L = R2L_df.label
X_U2R = U2R_df.drop('label',axis=1)
Y_U2R = U2R_df.label

# test set
X_DoS_test = DoS_df_test.drop('label',axis=1)
Y_DoS_test = DoS_df_test.label
X_Probe_test = Probe_df_test.drop('label',axis=1)
Y_Probe_test = Probe_df_test.label
X_R2L_test = R2L_df_test.drop('label',axis=1)
Y_R2L_test = R2L_df_test.label
X_U2R_test = U2R_df_test.drop('label',axis=1)
Y_U2R_test = U2R_df_test.label

In [21]:
print("test X:")
print(X_DoS_test.shape)
print(X_Probe_test.shape)
print(X_R2L_test.shape)
print(X_U2R_test.shape)
print()
print("test Y:")
print(Y_DoS_test.shape)
print(Y_Probe_test.shape)
print(Y_R2L_test.shape)
print(Y_U2R_test.shape)

test X:
(17171, 122)
(12132, 122)
(12596, 122)
(9778, 122)

test Y:
(17171,)
(12132,)
(12596,)
(9778,)


In [22]:
colNames=list(X_DoS)
colNames_test=list(X_DoS_test)

print(len(colNames))
print(colNames)
print()
print(len(colNames_test))
print(colNames_test)



122
['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo',

### Normalizing the features

In [23]:
# train set
scaler1 = preprocessing.StandardScaler().fit(X_DoS)
X_DoS=scaler1.transform(X_DoS)
scaler2 = preprocessing.StandardScaler().fit(X_Probe)
X_Probe=scaler2.transform(X_Probe)
scaler3 = preprocessing.StandardScaler().fit(X_R2L)
X_R2L=scaler3.transform(X_R2L)
scaler4 = preprocessing.StandardScaler().fit(X_U2R)
X_U2R=scaler4.transform(X_U2R)

# test data
scaler5 = preprocessing.StandardScaler().fit(X_DoS_test)
X_DoS_test=scaler5.transform(X_DoS_test)
scaler6 = preprocessing.StandardScaler().fit(X_Probe_test)
X_Probe_test=scaler6.transform(X_Probe_test)
scaler7 = preprocessing.StandardScaler().fit(X_R2L_test)
X_R2L_test=scaler7.transform(X_R2L_test)
scaler8 = preprocessing.StandardScaler().fit(X_U2R_test)
X_U2R_test=scaler8.transform(X_U2R_test)

In [24]:
print(X_DoS.std(axis=0))

X_Probe.std(axis=0)
X_R2L.std(axis=0)
X_U2R.std(axis=0)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1.]


array([1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1.])

# ANNOVA F-test

In this section, we use ANOVA F-test to perform feature selection for our classification task. The F-test helps us evaluate which features have the most significant relationship with the target variable. Specifically, we apply the test to select the top 10% of features from the original dataset for four categories: DoS, Probe, R2L, and U2R.

In [25]:
from sklearn.feature_selection import SelectPercentile, f_classif

np.seterr(divide='ignore', invalid='ignore')

# This function selects the top 10% of features based on the ANOVA F-test score (f_classif is the scoring function).
selector=SelectPercentile(f_classif, percentile=10) 

In [26]:
# Fits the selector on the DoS dataset (X_DoS and Y_DoS), keeping only the top 10% of features.
X_newDoS = selector.fit_transform(X_DoS,Y_DoS)
print(X_newDoS.shape)

true_DoS=selector.get_support() # Returns a boolean mask indicating which features were selected.
newcolindex_DoS=[i for i, x in enumerate(true_DoS) if x] # Extracts the indices of the selected features.
newcolname_DoS=list( colNames[i] for i in newcolindex_DoS ) # Retrieves the names of the selected features using the colNames list.
print(newcolname_DoS)

(113270, 13)
['logged_in', 'count', 'serror_rate', 'srv_serror_rate', 'same_srv_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'service_http', 'flag_S0', 'flag_SF']


c:\Users\Pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 16  44  63  66  68  86 114] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


In [27]:
X_newProbe = selector.fit_transform(X_Probe,Y_Probe)
print(X_newProbe.shape)

true_Probe=selector.get_support()
newcolindex_Probe=[i for i, x in enumerate(true_Probe) if x]
newcolname_Probe=list( colNames[i] for i in newcolindex_Probe )
print(newcolname_Probe)

(78999, 13)
['logged_in', 'rerror_rate', 'srv_rerror_rate', 'dst_host_srv_count', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Protocol_type_icmp', 'service_eco_i', 'service_private', 'flag_SF']


c:\Users\Pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 4 16] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


In [28]:
X_newR2L = selector.fit_transform(X_R2L,Y_R2L)
print(X_newR2L.shape)

true_R2L=selector.get_support()
newcolindex_R2L=[i for i, x in enumerate(true_R2L) if x]
newcolname_R2L=list( colNames[i] for i in newcolindex_R2L)
print(newcolname_R2L)

(68338, 13)
['src_bytes', 'dst_bytes', 'hot', 'num_failed_logins', 'is_guest_login', 'dst_host_srv_count', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'service_ftp', 'service_ftp_data', 'service_http', 'service_imap4', 'flag_RSTO']


c:\Users\Pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  4  16  43  44  46  47  48  49  50  51  54  57  58  62  63  64  66  67
  68  70  71  72  73  74  76  77  78  79  80  81  82  83  86  87  89  92
  93  96  98  99 100 107 108 109 110 114] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


In [29]:
X_newU2R = selector.fit_transform(X_U2R,Y_U2R)
print(X_newU2R.shape)

true_U2R=selector.get_support()
newcolindex_U2R=[i for i, x in enumerate(true_U2R) if x]
newcolname_U2R=list( colNames[i] for i in newcolindex_U2R)
print(newcolname_U2R)

(67395, 13)
['urgent', 'hot', 'root_shell', 'num_file_creations', 'num_shells', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'service_ftp_data', 'service_http', 'service_telnet']


c:\Users\Pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  4  16  43  44  46  47  48  49  50  51  54  57  58  62  63  64  66  67
  68  70  71  72  73  74  75  76  77  78  79  80  81  82  83  86  87  89
  92  93  96  98  99 100 107 108 109 110 114] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


In [30]:
print('Features selected for DoS:',newcolname_DoS, "\n")
print('Features selected for Probe:',newcolname_Probe, "\n")
print('Features selected for R2L:',newcolname_R2L, "\n")
print('Features selected for U2R:',newcolname_U2R, "\n")

Features selected for DoS: ['logged_in', 'count', 'serror_rate', 'srv_serror_rate', 'same_srv_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'service_http', 'flag_S0', 'flag_SF'] 

Features selected for Probe: ['logged_in', 'rerror_rate', 'srv_rerror_rate', 'dst_host_srv_count', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Protocol_type_icmp', 'service_eco_i', 'service_private', 'flag_SF'] 

Features selected for R2L: ['src_bytes', 'dst_bytes', 'hot', 'num_failed_logins', 'is_guest_login', 'dst_host_srv_count', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'service_ftp', 'service_ftp_data', 'service_http', 'service_imap4', 'flag_RSTO'] 

Features selected for U2R: ['urgent', 'hot', 'root_shell', 'num_file_creations', 'num_shells', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_ho

# IDS

To make best IDS model several machine classifier are selected:
- Logistic Regression (LR): A linear model for classification.
- Gaussian Naive Bayes (NB): A probabilistic classifier based on Bayes' theorem.
- Decision Tree (DT): A tree-based model that splits data into classes.
- AdaBoost (AB): An ensemble learning method using boosting.
- Random Forest (RF): An ensemble of decision trees using bagging.

At the end we build a Voting Classifier using these machine learning models (Logistic Regression, Naive Bayes, Decision Tree, AdaBoost, and Random Forest). By combining these models, we aim to improve the predictive performance for detecting each attacks. 

## DoS Attack

In the Probe attack classification models, the labels were originally 0 and 2. Since the binary method could not be used in this case, the labels were initially updated to 0 and 1, and all models were run. 

Afterward, in the R2L classification the label distribution in the dataset was examined, and considering the imbalance, the average values were changed to **weighted**. It was observed that the models performed better under these conditions. 

To try again, the attack label values were left as 0 and 2 in the Probe attack classification section, and the model score average continued to use the weighted parameter. Here too, an increase in the scores was observed. Then, the weighted approach was tested for the DoS and U2R datasets as well, and again, an improvement in model performance was noted.

**As a result because of the imbalanced of the datasets the average values were continued with the weighted**.

In [31]:
DoS_df["label"].value_counts()

label
0    67343
1    45927
Name: count, dtype: int64

In [32]:
# DoS_df contains both normal data and data related to DoS attacks. 
# We select the features that best represent DoS attacks from this dataset to train models 
# that can accurately detect and classify DoS attacks.

new_DoS_dataset_annova=DoS_df [newcolname_DoS] 
new_DoS_testdataset_annova=DoS_df_test [newcolname_DoS]

X_DoS = new_DoS_dataset_annova.iloc[:,0:new_DoS_dataset_annova.shape[1]] # inputs
Y_DoS = DoS_df['label'] # outputs
C_DoS = DoS_df_test['label'] # outputs for testing
T_DoS = new_DoS_testdataset_annova.iloc[:,0:new_DoS_dataset_annova.shape[1]] # inputs for testing


scaler_DoS = Normalizer().fit(X_DoS)
trainX_DoS = scaler_DoS.transform(X_DoS)

scaler_DoS = Normalizer().fit(T_DoS)
testT_DoS = scaler_DoS.transform(T_DoS)

traindata_DoS = np.array(trainX_DoS)
trainlabel_DoS = np.array(Y_DoS)

testdata_DoS = np.array(testT_DoS)
testlabel_DoS = np.array(C_DoS)

print(traindata_DoS.shape)
print(testdata_DoS.shape)
print()
print(X_DoS.shape)
print(Y_DoS.shape)
print(C_DoS.shape)
print(T_DoS.shape)
# 13 selected DoS's features

(113270, 13)
(17171, 13)

(113270, 13)
(113270,)
(17171,)
(17171, 13)


In [33]:
print("------------Logistic Regression--------------")

modelLR = LogisticRegression(class_weight="balanced")
modelLR.fit(traindata_DoS, trainlabel_DoS)

expected = testlabel_DoS
np.savetxt('classical/DoSexpected.txt', expected, fmt='%01d')
predicted = modelLR.predict(testdata_DoS)
proba = modelLR.predict_proba(testdata_DoS)

np.savetxt('classical/DoSpredictedlabelLR.txt', predicted, fmt='%01d')
np.savetxt('classical/DoSpredictedprobaLR.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Logistic Regression--------------
Accuracy:  0.816
Precision:  0.824
Recall:  0.816
F1 score:  0.811


In [34]:
print("------------Gaussian Naive Bayes--------------")

model = GaussianNB()
model.fit(traindata_DoS, trainlabel_DoS)

expected = testlabel_DoS
predicted = model.predict(testdata_DoS)
proba = model.predict_proba(testdata_DoS)

np.savetxt('classical/DoSpredictedlabelNB.txt', predicted, fmt='%01d')
np.savetxt('classical/DoSpredictedprobaNB.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Gaussian Naive Bayes--------------
Accuracy:  0.813
Precision:  0.814
Recall:  0.813
F1 score:  0.811


In [35]:
print("------------Decision Tree--------------")

modelDT = DecisionTreeClassifier(class_weight="balanced")
modelDT.fit(traindata_DoS, trainlabel_DoS)

expected = testlabel_DoS
predicted = modelDT.predict(testdata_DoS)
proba = modelDT.predict_proba(testdata_DoS)

np.savetxt('classical/DoSpredictedlabelDT.txt', predicted, fmt='%01d')
np.savetxt('classical/DoSpredictedprobaDT.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")


print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Decision Tree--------------
Accuracy:  0.845
Precision:  0.853
Recall:  0.845
F1 score:  0.841


In [36]:
print("------------Adaboost--------------")

modelAB = AdaBoostClassifier(n_estimators=100)
modelAB.fit(traindata_DoS, trainlabel_DoS)

expected = testlabel_DoS
predicted = modelAB.predict(testdata_DoS)
proba = modelAB.predict_proba(testdata_DoS)

np.savetxt('classical/DoSpredictedlabelAB.txt', predicted, fmt='%01d')
np.savetxt('classical/DoSpredictedprobaAB.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")


print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Adaboost--------------
Accuracy:  0.836
Precision:  0.847
Recall:  0.836
F1 score:  0.832


In [37]:
print("------------Random Forest--------------")

modelRF = RandomForestClassifier(n_estimators=100)
modelRF = modelRF.fit(traindata_DoS, trainlabel_DoS)

expected = testlabel_DoS
predicted = modelRF.predict(testdata_DoS)
proba = modelRF.predict_proba(testdata_DoS)
np.savetxt('classical/DoSpredictedlabelRF.txt', predicted, fmt='%01d')
np.savetxt('classical/DoSpredictedprobaRF.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")


print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Random Forest--------------
Accuracy:  0.829
Precision:  0.841
Recall:  0.829
F1 score:  0.824


The VotingClassifier combines all the above models.
Soft voting is used, meaning the classifier predicts the class label based on the average predicted probabilities of the models.

In [38]:
modelLR = LogisticRegression(class_weight="balanced")
modelNB = GaussianNB()
modelDT = DecisionTreeClassifier(class_weight="balanced")
modelAB = AdaBoostClassifier(n_estimators=100)
modelRF = RandomForestClassifier(n_estimators=100)

voting_clf_DoS = VotingClassifier(estimators=[
    ('LR', modelLR),
    ('NB', modelNB),
    ('DT', modelDT),
    ('Adaboost', modelAB),
    ('RF', modelRF)
], voting='soft')  

voting_clf_DoS.fit(traindata_DoS, trainlabel_DoS)

predicted = voting_clf_DoS.predict(testdata_DoS)
proba = voting_clf_DoS.predict_proba(testdata_DoS)

np.savetxt('classical/predictedlabelVotingDoS.txt', predicted, fmt='%01d')
np.savetxt('classical/predictedprobaVotingDoS.txt', proba)

expected = testlabel_DoS
y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")


print("Prediction Performance of Voting Classifier with DoS Dataset")
print("Accuracy:", ("%.3f" %accuracy))
print("Precision:", ("%.3f" %precision))
print("Recall:", ("%.3f" %recall))
print("F1 Score:", ("%.3f" %f1))

Prediction Performance of Voting Classifier with DoS Dataset
Accuracy: 0.826
Precision: 0.836
Recall: 0.826
F1 Score: 0.822


## Probe Attack

In [39]:
Probe_df["label"].value_counts()

label
0    67343
2    11656
Name: count, dtype: int64

In [40]:
# get the Probe feature from the train and test set
new_Probe_dataset_annova=Probe_df [newcolname_Probe]
new_Probe_testdataset_annova=Probe_df_test [newcolname_Probe]

X_Probe = new_Probe_dataset_annova.iloc[:,0:new_Probe_dataset_annova.shape[1]] # inputs
Y_Probe = Probe_df['label']# outputs
C_Probe = Probe_df_test['label'] # outputs for testing
T_Probe = new_Probe_testdataset_annova.iloc[:,0:new_Probe_dataset_annova.shape[1]] # inputs for testing

scaler_Probe = Normalizer().fit(X_Probe)
trainX_Probe = scaler_Probe.transform(X_Probe)

scaler_Probe = Normalizer().fit(T_Probe)
testT_Probe = scaler_Probe.transform(T_Probe)

traindata_Probe = np.array(trainX_Probe)
trainlabel_Probe = np.array(Y_Probe)

testdata_Probe = np.array(testT_Probe)
testlabel_Probe = np.array(C_Probe)

print(traindata_Probe.shape) # (78999, 13)
print(testdata_Probe.shape) # (12132, 13)


(78999, 13)
(12132, 13)


In [41]:
print("------------Logistic Regression--------------")

modelLR = LogisticRegression(class_weight="balanced")
modelLR.fit(traindata_Probe, trainlabel_Probe)

expected = testlabel_Probe
np.savetxt('classical/Probeexpected.txt', expected, fmt='%01d')
predicted = modelLR.predict(testdata_Probe)
proba = modelLR.predict_proba(testdata_Probe)

np.savetxt('classical/ProbepredictedlabelLR.txt', predicted, fmt='%01d')
np.savetxt('classical/ProbepredictedprobaLR.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Logistic Regression--------------
Accuracy:  0.880
Precision:  0.876
Recall:  0.880
F1 score:  0.868


c:\Users\Pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
print("------------Gaussian Naive Bayes--------------")

modelNB = GaussianNB()
modelNB.fit(traindata_Probe, trainlabel_Probe)
print(modelNB)

expected = testlabel_Probe
predicted = modelNB.predict(testdata_Probe)
proba = modelNB.predict_proba(testdata_Probe)

np.savetxt('classical/ProbepredictedlabelNB.txt', predicted, fmt='%01d')
np.savetxt('classical/ProbepredictedprobaNB.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Gaussian Naive Bayes--------------
GaussianNB()
Accuracy:  0.886
Precision:  0.883
Recall:  0.886
F1 score:  0.875


In [43]:
print("------------Decision Tree--------------")

modelDT = DecisionTreeClassifier(class_weight="balanced")
modelDT.fit(traindata_Probe, trainlabel_Probe)
print(modelDT)

expected = testlabel_Probe
predicted = modelDT.predict(testdata_Probe)
proba = modelDT.predict_proba(testdata_Probe)

np.savetxt('classical/ProbepredictedlabelDT.txt', predicted, fmt='%01d')
np.savetxt('classical/ProbepredictedprobaDT.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Decision Tree--------------
DecisionTreeClassifier(class_weight='balanced')
Accuracy:  0.884
Precision:  0.881
Recall:  0.884
F1 score:  0.873


In [44]:
print("------------Adaboost--------------")

modelAB = AdaBoostClassifier(n_estimators=100)
modelAB.fit(traindata_Probe, trainlabel_Probe)

expected = testlabel_Probe
predicted = modelAB.predict(testdata_Probe)
proba = modelAB.predict_proba(testdata_Probe)

np.savetxt('classical/ProbepredictedlabelAB.txt', predicted, fmt='%01d')
np.savetxt('classical/ProbepredictedprobaAB.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")


print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Adaboost--------------
Accuracy:  0.892
Precision:  0.889
Recall:  0.892
F1 score:  0.883


In [45]:
print("------------Random Forest--------------")
modelRF = RandomForestClassifier(n_estimators=100)
modelRF = modelRF.fit(traindata_Probe, trainlabel_Probe)

expected = testlabel_Probe
predicted = modelRF.predict(testdata_Probe)
proba = modelRF.predict_proba(testdata_Probe)
np.savetxt('classical/ProbepredictedlabelRF.txt', predicted, fmt='%01d')
np.savetxt('classical/ProbepredictedprobaRF.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Random Forest--------------
Accuracy:  0.888
Precision:  0.885
Recall:  0.888
F1 score:  0.878


In [46]:
modelLR = LogisticRegression(class_weight="balanced")
modelNB = GaussianNB()
modelDT = DecisionTreeClassifier(class_weight="balanced")
modelAB = AdaBoostClassifier(n_estimators=100)
modelRF = RandomForestClassifier(n_estimators=100)

voting_clf_Probe = VotingClassifier(estimators=[
    ('LR', modelLR),
    ('NB', modelNB),
    ('DT', modelDT),
    ('Adaboost', modelAB),
    ('RF', modelRF)
], voting='soft')  

voting_clf_Probe.fit(traindata_Probe, trainlabel_Probe)

predicted = voting_clf_Probe.predict(testdata_Probe)
proba = voting_clf_Probe.predict_proba(testdata_Probe)

np.savetxt('classical/ProbepredictedlabelVotingProbe.txt', predicted, fmt='%01d')
np.savetxt('classical/ProbepredictedprobaVotingProbe.txt', proba)

expected = testlabel_Probe
y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Prediction Performance of Voting Classifier with Probe Dataset")
print("Accuracy:", ("%.3f" %accuracy))
print("Precision:", ("%.3f" %precision))
print("Recall:", ("%.3f" %recall))
print("F1 Score:", ("%.3f" %f1))

c:\Users\Pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Prediction Performance of Voting Classifier with Probe Dataset
Accuracy: 0.889
Precision: 0.885
Recall: 0.889
F1 Score: 0.879


## R2L Attack

In [47]:
R2L_df["label"].value_counts()

label
0    67343
3      995
Name: count, dtype: int64

In [48]:
# get the R2L feature from the train and test set

new_R2L_dataset_annova=R2L_df [newcolname_R2L]
new_R2L_testdataset_annova=R2L_df_test [newcolname_R2L]

X_R2L = new_R2L_dataset_annova.iloc[:,0:new_R2L_dataset_annova.shape[1]] # inputs
Y_R2L = R2L_df['label'] # outputs
C_R2L = R2L_df_test['label'] # outputs for testing
T_R2L = new_R2L_testdataset_annova.iloc[:,0:new_R2L_dataset_annova.shape[1]] # inputs for testing

scaler_R2L = Normalizer().fit(X_R2L)
trainX_R2L = scaler_R2L.transform(X_R2L)

scaler_R2L = Normalizer().fit(T_R2L)
testT_R2L = scaler_R2L.transform(T_R2L)


traindata_R2L = np.array(trainX_R2L)
trainlabel_R2L = np.array(Y_R2L)

testdata_R2L = np.array(testT_R2L)
testlabel_R2L = np.array(C_R2L)

print(traindata_R2L.shape)
print(testdata_R2L.shape)

(68338, 13)
(12596, 13)


In [49]:
print("------------Logistic Regression--------------")

modelLR = LogisticRegression(class_weight="balanced")
modelLR.fit(traindata_R2L, trainlabel_R2L)

# make predictions
expected = testlabel_R2L
np.savetxt('classical/R2Lexpected.txt', expected, fmt='%01d')
predicted = modelLR.predict(testdata_R2L)
proba = modelLR.predict_proba(testdata_R2L)

np.savetxt('classical/R2LpredictedlabelLR.txt', predicted, fmt='%01d')
np.savetxt('classical/R2LpredictedprobaLR.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Logistic Regression--------------
Accuracy:  0.723
Precision:  0.665
Recall:  0.723
F1 score:  0.685


In [50]:
print("-----------------Gaussian Naive Bayes-----------------")

modelNB = GaussianNB()
modelNB.fit(traindata_R2L, trainlabel_R2L)
print(modelNB)
# make predictions
expected = testlabel_R2L
predicted = modelNB.predict(testdata_R2L)
proba = modelNB.predict_proba(testdata_R2L)

np.savetxt('classical/R2LpredictedlabelNB.txt', predicted, fmt='%01d')
np.savetxt('classical/R2LpredictedprobaNB.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

-----------------Gaussian Naive Bayes-----------------
GaussianNB()
Accuracy:  0.850
Precision:  0.851
Recall:  0.850
F1 score:  0.828


In [51]:
print("-----------------Decision Tree-----------------")

modelDT = DecisionTreeClassifier(class_weight="balanced")
modelDT.fit(traindata_R2L, trainlabel_R2L)
print(modelDT)

expected = testlabel_R2L
predicted = modelDT.predict(testdata_R2L)
proba = modelDT.predict_proba(testdata_R2L)

np.savetxt('classical/R2LpredictedlabelDT.txt', predicted, fmt='%01d')
np.savetxt('classical/R2LpredictedprobaDT.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

-----------------Decision Tree-----------------
DecisionTreeClassifier(class_weight='balanced')
Accuracy:  0.805
Precision:  0.843
Recall:  0.805
F1 score:  0.743


In [52]:
print("------------ Adaboost --------------")

modelAB = AdaBoostClassifier(n_estimators=100)
modelAB.fit(traindata_R2L, trainlabel_R2L)

expected = testlabel_R2L
predicted = modelAB.predict(testdata_R2L)
proba = modelAB.predict_proba(testdata_R2L)

np.savetxt('classical/R2LpredictedlabelAB.txt', predicted, fmt='%01d')
np.savetxt('classical/R2LpredictedprobaAB.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------ Adaboost --------------
Accuracy:  0.774
Precision:  0.820
Recall:  0.774
F1 score:  0.679


In [53]:
print("------------Random Forest--------------")

modelRF = RandomForestClassifier(n_estimators=100)
modelRF = modelRF.fit(traindata_R2L, trainlabel_R2L)

expected = testlabel_R2L
predicted = modelRF.predict(testdata_R2L)
proba = modelRF.predict_proba(testdata_R2L)
np.savetxt('classical/R2LpredictedlabelRF.txt', predicted, fmt='%01d')
np.savetxt('classical/R2LpredictedprobaRF.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

------------Random Forest--------------
Accuracy:  0.777
Precision:  0.824
Recall:  0.777
F1 score:  0.686


In [54]:
modelLR = LogisticRegression(class_weight="balanced")
modelNB = GaussianNB()
modelDT = DecisionTreeClassifier(class_weight="balanced")
modelAB = AdaBoostClassifier(n_estimators=100)
modelRF = RandomForestClassifier(n_estimators=100)

voting_clf_R2L = VotingClassifier(estimators=[
    ('LR', modelLR),
    ('NB', modelNB),
    ('DT', modelDT),
    ('Adaboost', modelAB),
    ('RF', modelRF)
], voting='soft')  

voting_clf_R2L.fit(traindata_R2L, trainlabel_R2L)

predicted = voting_clf_R2L.predict(testdata_R2L)
proba = voting_clf_R2L.predict_proba(testdata_R2L)

np.savetxt('classical/R2LpredictedlabelVoting.txt', predicted, fmt='%01d')
np.savetxt('classical/R2LpredictedprobaVoting.txt', proba)

expected = testlabel_R2L
y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted") 
precision = precision_score(y_train1, y_pred , average="weighted") 
f1 = f1_score(y_train1, y_pred, average="weighted") 


print("Prediction Performance of Voting Classifier with R2L Dataset")
print("Accuracy:", ("%.3f" %accuracy))
print("Precision:", ("%.3f" %precision))
print("Recall:", ("%.3f" %recall))
print("F1 Score:", ("%.3f" %f1))

Prediction Performance of Voting Classifier with R2L Dataset
Accuracy: 0.808
Precision: 0.839
Recall: 0.808
F1 Score: 0.750


## U2R

In [62]:
U2R_df["label"].value_counts()

label
0    67343
4       52
Name: count, dtype: int64

In [55]:
# get the U2R feature from the train and test set

new_U2R_dataset_annova=U2R_df [newcolname_U2R]
new_U2R_testdataset_annova=U2R_df_test [newcolname_U2R]

X_U2R = new_U2R_dataset_annova.iloc[:,0:new_U2R_dataset_annova.shape[1]] # inputs
Y_U2R = U2R_df['label']# outputs
C_U2R = U2R_df_test['label'] # outputs for testing
T_U2R = new_U2R_testdataset_annova.iloc[:,0:new_U2R_dataset_annova.shape[1]] # inputs for testing

scaler_U2R = Normalizer().fit(X_U2R)
trainX_U2R = scaler_U2R.transform(X_U2R)

scaler_U2R = Normalizer().fit(T_U2R)
testT_U2R = scaler_U2R.transform(T_U2R)


traindata_U2R = np.array(trainX_U2R)
trainlabel_U2R = np.array(Y_U2R)

testdata_U2R = np.array(testT_U2R)
testlabel_U2R = np.array(C_U2R)

print(traindata_U2R.shape)
print(testdata_U2R.shape)

(67395, 13)
(9778, 13)


In [56]:
print("-----------------Logistic Regression-----------------")

modelLR = LogisticRegression(class_weight="balanced")
modelLR.fit(traindata_U2R, trainlabel_U2R)

expected = testlabel_U2R
np.savetxt('classical/U2Rexpected.txt', expected, fmt='%01d')
predicted = modelLR.predict(testdata_U2R)
proba = modelLR.predict_proba(testdata_U2R)

np.savetxt('classical/U2RpredictedlabelLR.txt', predicted, fmt='%01d')
np.savetxt('classical/U2RpredictedprobaLR.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

-----------------Logistic Regression-----------------
Accuracy:  0.960
Precision:  0.991
Recall:  0.960
F1 score:  0.974


c:\Users\Pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
print("-----------------Gaussian Naive Bayes-----------------")

modelNB = GaussianNB()
modelNB.fit(traindata_U2R, trainlabel_U2R)
print(modelNB)

expected = testlabel_U2R
predicted = modelNB.predict(testdata_U2R)
proba = modelNB.predict_proba(testdata_U2R)

np.savetxt('classical/U2RpredictedlabelNB.txt', predicted, fmt='%01d')
np.savetxt('classical/U2RpredictedprobaNB.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

-----------------Gaussian Naive Bayes-----------------
GaussianNB()
Accuracy:  0.992
Precision:  0.992
Recall:  0.992
F1 score:  0.992


In [58]:
print("-----------------Decision Tree-----------------")

modelDT = DecisionTreeClassifier(class_weight="balanced")
modelDT.fit(traindata_U2R, trainlabel_U2R)
print(modelDT)

expected = testlabel_U2R
predicted = modelDT.predict(testdata_U2R)
proba = modelDT.predict_proba(testdata_U2R)

np.savetxt('classical/U2RpredictedlabelDT.txt', predicted, fmt='%01d')
np.savetxt('classical/U2RpredictedprobaDT.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

-----------------Decision Tree-----------------


DecisionTreeClassifier(class_weight='balanced')
Accuracy:  0.991
Precision:  0.989
Recall:  0.991
F1 score:  0.990


In [59]:
print("-----------------Adaboost-----------------")

modelAB = AdaBoostClassifier(n_estimators=100)
modelAB.fit(traindata_U2R, trainlabel_U2R)

expected = testlabel_U2R
predicted = modelAB.predict(testdata_U2R)
proba = modelAB.predict_proba(testdata_U2R)

np.savetxt('classical/U2RpredictedlabelAB.txt', predicted, fmt='%01d')
np.savetxt('classical/U2RpredictedprobaAB.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))

-----------------Adaboost-----------------
Accuracy:  0.994
Precision:  0.994
Recall:  0.994
F1 score:  0.991


In [60]:
print("-----------------Random Forest-----------------")

modelRF = RandomForestClassifier(n_estimators=100)
modelRF = modelRF.fit(traindata_U2R, trainlabel_U2R)

expected = testlabel_U2R
predicted = modelRF.predict(testdata_U2R)
proba = modelRF.predict_proba(testdata_U2R)
np.savetxt('classical/U2RpredictedlabelRF.txt', predicted, fmt='%01d')
np.savetxt('classical/U2RpredictedprobaRF.txt', proba)

y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Accuracy: ", ("%.3f" %accuracy))
print("Precision: ", ("%.3f" %precision))
print("Recall: ", ("%.3f" %recall))
print("F1 score: ", ("%.3f" %f1))


-----------------Random Forest-----------------
Accuracy:  0.994
Precision:  0.994
Recall:  0.994
F1 score:  0.991


In [61]:
modelLR = LogisticRegression(class_weight="balanced")
modelNB = GaussianNB()
modelDT = DecisionTreeClassifier(class_weight="balanced")
modelAB = AdaBoostClassifier(n_estimators=100)
modelRF = RandomForestClassifier(n_estimators=100)

voting_clf_U2R = VotingClassifier(estimators=[
    ('LR', modelLR),
    ('NB', modelNB),
    ('DT', modelDT),
    ('Adaboost', modelAB),
    ('RF', modelRF)
], voting='soft') 

voting_clf_U2R.fit(traindata_U2R, trainlabel_U2R)

predicted = voting_clf_U2R.predict(testdata_U2R)
proba = voting_clf_U2R.predict_proba(testdata_U2R)

np.savetxt('classical/U2RpredictedlabelVotingU2R.txt', predicted, fmt='%01d')
np.savetxt('classical/U2RpredictedprobaVotingU2R.txt', proba)

expected = testlabel_U2R
y_train1 = expected
y_pred = predicted
accuracy = accuracy_score(y_train1, y_pred)
recall = recall_score(y_train1, y_pred , average="weighted")
precision = precision_score(y_train1, y_pred , average="weighted")
f1 = f1_score(y_train1, y_pred, average="weighted")

print("Prediction Performance of Voting Classifier with U2R Dataset")
print("Accuracy:", ("%.3f" %accuracy))
print("Precision:", ("%.3f" %precision))
print("Recall:", ("%.3f" %recall))
print("F1 Score:", ("%.3f" %f1))

c:\Users\Pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Prediction Performance of Voting Classifier with U2R Dataset
Accuracy: 0.993
Precision: 0.992
Recall: 0.993
F1 Score: 0.992


# Experiment through the dataset

In [63]:
import random

def predictAttack(dataset, model_type):
  if len(dataset) == 0:
    return "Dataset is empty!"

  row_num = dataset.shape[0]
  random_row = random.randint(0, row_num - 1)
  # selected_row = dataset[random_row].reshape(1, -1)
  selected_row = dataset[random_row].reshape(1, -1)
  predict = None
  expected = None

  if model_type == "voting_clf_DoS":
    predict = voting_clf_DoS.predict(selected_row)
    expected = testlabel_DoS[random_row]

  elif model_type == "voting_clf_Probe":
    predict = voting_clf_Probe.predict(selected_row)
    expected = testlabel_Probe[random_row]

  elif model_type == "voting_clf_R2L":
    predict = voting_clf_R2L.predict(selected_row)
    expected = testlabel_R2L[random_row]

  elif model_type == "voting_clf_U2R":
    predict = voting_clf_U2R.predict(selected_row)
    expected = testlabel_U2R[random_row]
  else:
    return "Unknown model type."

  return f"The result of the selected model for the {random_row}th row in the given data set is {predict}.\n For this row expected label is {expected}"


In [85]:
# Datasets
# testdata_DoS
# testdata_Probe
# testdata_R2L
# testdata_U2R

# Models
# voting_clf_DoS
# voting_clf_Probe
# voting_clf_R2L
# voting_clf_U2R

print("DoS")
result = predictAttack(testdata_DoS, "voting_clf_DoS")
print(result)
print()
print("Probe")
result = predictAttack(testdata_Probe, "voting_clf_Probe")
print(result)
print()
print("R2L")
result = predictAttack(testdata_R2L, "voting_clf_R2L")
print(result)
print()
print("U2R")
result = predictAttack(testdata_U2R, "voting_clf_U2R")
print(result)

DoS
The result of the selected model for the 13556th row in the given data set is [1].
 For this row expected label is 1

Probe
The result of the selected model for the 1918th row in the given data set is [2].
 For this row expected label is 0

R2L
The result of the selected model for the 11706th row in the given data set is [0].
 For this row expected label is 3

U2R
The result of the selected model for the 4753th row in the given data set is [0].
 For this row expected label is 0
